langchain → é o framework que facilita criar agentes de IA, chains, ferramentas etc.


In [4]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
#.env → arquivo para guardar chaves/senhas/configurações.
# Carregar variáveis do arquivo .env
load_dotenv()


True

In [6]:
#pega a chave da API do google
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [7]:
# Criar o LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

**Conexão com o Gemini:**

- **LLM** = *Large Language Model*, ou *Modelo de Linguagem Grande*. É o mesmo conceito do GPT-4 ou ChatGPT, mas do Google (Gemini).

- **ChatGoogleGenerativeAI** é a classe (biblioteca instalada anteriormente) do LangChain que cria um “agente de chat” usando esse modelo.

- `model="gemini-2.5-flash"` → você está escolhendo **qual modelo do Gemini** quer usar.

- `temperature=0.0` → controla a **aleatoriedade da resposta** (0 = respostas mais previsíveis).

- `api_key=GOOGLE_API_KEY` → você está passando sua **chave** para o modelo aceitar requisições.


In [8]:
# Pergunta ao LLM
resp_test = llm.invoke("Quem é você? Seja criativo.")

In [11]:
# Mostra a resposta; .content → mostra apenas conteudo
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

- **Prompt do sistema**: instrui o modelo a atuar como um triador de Service Desk, retornando **apenas um JSON** com os campos `decisao`, `urgencia` e `campos_faltantes`, seguindo regras específicas de classificação e prioridade.


In [10]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

Definir um modelo de saída estruturado usando Pydantic - que valida dados:

BaseModel → classe base do Pydantic para criar modelos de dados.

Field → permite definir valores padrão ou regras para os campos.

Literal → garante que o campo só aceite valores específicos.

List → indica que um campo é uma lista.

In [12]:
from pydantic import BaseModel, Field
from typing import Literal, List

In [ ]:
#Classe de saída estruturada que se espera da IA(limita a saida)
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)


In [14]:
#Criando uma nova instância do LLM especificamente para a triagem

llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
#Importa classes que representam mensagens para o modelo para diferenciar as duas partes da conversa:
#SystemMessage → instruções do sistema (prompt do sistema).
#HumanMessage → mensagens enviadas pelo usuário.

from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
#Dict = dicionário (chave: valor)
#List = lista (coleção de itens)
#Literal = literal (valor fixo, permitido apenas alguns)

from typing import Dict, List, Literal


In [ ]:
#Cria um “chain” estruturado, dizendo que o LLM deve produzir saídas no formato definido pelo modelo TriagemOut
#fluxo de triagem com o llm_triagem
triagem_chain = llm_triagem.with_structured_output(TriagemOut)


In [ ]:
#Função de triagem 
def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()


- A função **triagem** recebe uma mensagem do usuário, envia essa mensagem junto com o **prompt do sistema** para o LLM, e recebe uma resposta.  
- Essa resposta é validada pelo modelo **TriagemOut**, garantindo que tenha os campos **decisao**, **urgencia** e **campos_faltantes**.  
- Por fim, a função retorna a resposta **como um dicionário (Dict)**, pronto para ser usado no código.


In [20]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [21]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre política interna']}

